<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#버전-2까지-수행하고-난-후의-개선사항" data-toc-modified-id="버전-2까지-수행하고-난-후의-개선사항-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>버전 2까지 수행하고 난 후의 개선사항</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#데이터-로드-및-FE-함수" data-toc-modified-id="데이터-로드-및-FE-함수-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>데이터 로드 및 FE 함수</a></span></li></ul></li></ul></li></ul></div>

# 버전 2까지 수행하고 난 후의 개선사항
- 날씨 데이터 사용의 효과가 그닥.. -> 날씨 데이터 그만파자.. 오히려 빼는게 좋을 수도.. -> 자기상관변수 증감률 데이터 FE하기!
- 모든 데이터를 다 사용했을 때가 Test 데이터에 대해 리더보드 결과는 더 좋음.. -> 모든 데이터 다 사용하기
- 다른 모델들 사용 -> 여러가지 모델...뭘 사용할까!?
- 함수화 할것
    - 데이터 로드 및 FE 함수
    - 모델링 후 학습, 검증 결과 출력하는 함수
    - 모델링 학습 후 최종 Test 출력하는 함수
    - 제출하는 csv 파일 만드는 함수

### 데이터 로드 및 FE 함수
- 날씨 데이터는 빼고 시도..
- 증감률 추가해야 함

In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import pymysql
import warnings
warnings.filterwarnings(action='ignore')


def load_datasets(path) -> pd.DataFrame:
    train = pd.read_csv(os.path.join(path, '한국가스공사_시간별 공급량_20181231.csv'),
                        encoding='cp949')
    test = pd.read_csv(os.path.join(path, 'test.csv'))
    # test 데이터 전처리
    test['연월일'] = test['일자|시간|구분'].str.split(' ', expand=True)[0]
    test['시간'] = test['일자|시간|구분'].str.split(' ', expand=True)[1].astype(int)
    test['구분'] = test['일자|시간|구분'].str.split(' ', expand=True)[2]
    del test['일자|시간|구분']

    data = pd.concat([train, test], axis=0)
    data['연월일'] = pd.to_datetime(data['연월일'])
    return data

def load_weather() -> pd.DataFrame:
    db = pymysql.connect(host='localhost', port=3306, user='younghun', password='watson1259',
                        db='dacon_gas_weather_db', charset='utf8')
    cursor = db.cursor()
    sql = "SELECT datetime, avg_temp, min_temp, max_temp,\
              NULLIF(sum_rain, '') as sum_rain, avg_wind, avg_humid,\
              sum_gsr, NULLIF(ddmefs, '') as ddmefs, avg_ts \
              FROM weather ORDER BY datetime"
    
    weather = pd.read_sql(sql, db)
    weather = weather.fillna(0.)
    
    return weather
    
def merge_gas_weather(path):
    gas = load_datasets(path)
    weather = load_weather()
    gas_weather = gas.merge(weather, how='left', left_on='연월일', right_on='datetime')
    del gas_weather['datetime']
    
    return gas_weather

def make_datetime_vars(data) -> pd.DataFrame:
    data['year'] = data['연월일'].dt.year
    data['month'] = data['연월일'].dt.month
    data['day'] = data['연월일'].dt.day
    data['week_no'] = data['연월일'].dt.strftime("%V").astype(int)
    data['dayofweek'] = data['연월일'].dt.day_name()
    data['weekend_yn'] = np.where(data['dayofweek'].isin(['Saturday', 'Sunday']), 1, 0)
    data['dayofyear'] = data['연월일'].dt.dayofyear

    return data


def change_dates(row):
    if row['시간'] == 24:
        row['연월일'] += pd.DateOffset(days=1)
        row['시간'] = 0
    return row


def extract_need_data(df):
    """ 연월일 날짜 포맷 바꾸고 매년 01-01~03-31 기간 데이터만 추출"""
    df = df.apply(change_dates, axis=1)
    df['시간'] = df['시간'].apply(lambda x: str(x) if x >= 10 else '0' + str(x))
    df['연월일'] = df['연월일'].astype(str)
    df['datetime'] = df['연월일'] + ' ' + df['시간']
    df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H')

    df['prev_datetime'] = df['datetime'] - pd.DateOffset(years=1)

    return df


def make_autocorr_vars(df, gubun='A'):
    sub_df = df[df['구분'] == gubun]
    # 일년 전 그 시간대 공급량
    sub_df['prev_year_공급량'] = sub_df.groupby([sub_df['datetime'].dt.month, sub_df['datetime'].dt.day,
                                          sub_df['datetime'].dt.hour])['공급량'].shift()
#     # 이년 전 공급량 파생변수 생성
#     sub_df['prev_two_year_공급량'] = sub_df.groupby([sub_df['datetime'].dt.month, sub_df['datetime'].dt.day,
#                                               sub_df['datetime'].dt.hour])['공급량'].shift(2)
    # 일년 전 평균 공급량
    sub_df['prev_year'] = sub_df['prev_datetime'].dt.year
    prev_year_mean_df = sub_df.groupby([sub_df['datetime'].dt.year])['공급량'].mean()
    prev_year_mean_dict = dict(zip(prev_year_mean_df.index, prev_year_mean_df.values.reshape(-1, )))
    sub_df['prev_year_avg_공급량'] = sub_df['prev_year'].map(prev_year_mean_dict)
    # 일년 전 월별 평균 공급량
    prev_month_mean_df = sub_df.groupby([sub_df['datetime'].dt.year, sub_df['datetime'].dt.month])['공급량'].mean()
    prev_month_mean_dict = dict(zip(prev_month_mean_df.index, prev_month_mean_df.values.reshape(-1, )))
    sub_df['prev_year_month'] = tuple(zip(sub_df['prev_year'].values.reshape(-1, ),
                                        sub_df['prev_datetime'].dt.month.values.reshape(-1, )))
    sub_df['prev_month_avg_공급량'] = sub_df['prev_year_month'].map(prev_month_mean_dict)
    # 일년 전 월-일자별 평균 공급량
    prev_month_day_mean_df = sub_df.groupby([sub_df['datetime'].dt.year,
                                           sub_df['datetime'].dt.month,
                                           sub_df['datetime'].dt.day])['공급량'].mean()
    prev_month_day_mean_dict = dict(zip(prev_month_day_mean_df.index, prev_month_day_mean_df.values.reshape(-1, )))

    sub_df['prev_year_month_day'] = tuple(zip(sub_df['prev_year'].values.reshape(-1, ),
                                            sub_df['prev_datetime'].dt.month.values.reshape(-1, ),
                                            sub_df['prev_datetime'].dt.day.values.reshape(-1, )))
    sub_df['prev_month_day_avg_공급량'] = sub_df['prev_year_month_day'].map(prev_month_day_mean_dict)
    # 일년 전 월-시간 별 평균 공급량
    prev_month_hour_mean_df = sub_df.groupby([sub_df['datetime'].dt.year,
                                            sub_df['datetime'].dt.month,
                                            sub_df['datetime'].dt.hour])['공급량'].mean()
    prev_month_hour_mean_dict = dict(zip(prev_month_hour_mean_df.index, prev_month_hour_mean_df.values.reshape(-1, )))

    sub_df['prev_year_month_hour'] = tuple(zip(sub_df['prev_year'].values.reshape(-1, ),
                                             sub_df['prev_datetime'].dt.month.values.reshape(-1, ),
                                             sub_df['prev_datetime'].dt.hour.values.reshape(-1, )))
    sub_df['prev_month_hour_avg_공급량'] = sub_df['prev_year_month_hour'].map(prev_month_hour_mean_dict)
    # 일년 전 그 날의 기상 변수 FE
    weather_cols = ['avg_temp','min_temp','max_temp','sum_rain','avg_wind','avg_humid','sum_gsr','ddmefs','avg_ts']
    for col in weather_cols:
        sub_df[f'prev_year_{col}'] = sub_df.groupby([sub_df['datetime'].dt.month, sub_df['datetime'].dt.day,
                                                  sub_df['datetime'].dt.hour])[col].shift()
        sub_df[f'prev_year_{col}'] = pd.to_numeric(sub_df[f'prev_year_{col}'], downcast="float")

    # 필요한 칼럼들만 추출
#     used_cols = ['연월일', '시간', 'datetime', '구분', 'prev_year_공급량',
#                  'prev_two_year_공급량', 'prev_year_avg_공급량', 'prev_month_avg_공급량', 'prev_month_day_avg_공급량',
#                  'prev_month_hour_avg_공급량', '공급량']
    used_cols = ['연월일', '시간', 'datetime', '구분', 'prev_year_공급량', 'prev_year_avg_공급량',
                 'prev_month_avg_공급량', 'prev_month_day_avg_공급량', 'prev_month_hour_avg_공급량']
    weather_cols = [f'prev_year_{col}' for col in weather_cols]
    used_cols += weather_cols + ['공급량']
    sub_df = sub_df[used_cols]
    return sub_df


def fe_autocorr_vars(df):
    final_df = pd.DataFrame()
    gubun_cols = df['구분'].unique()
    for gubun in tqdm(gubun_cols):
        sub_df = make_autocorr_vars(df, gubun=gubun)
        final_df = pd.concat([final_df, sub_df], axis=0)
    return final_df

def change_dates_adversely(row):
    if row['datetime'].hour == 0:
        row['연월일'] -= pd.DateOffset(days=1)
        row['시간'] = 24
    return row

def change_date_format(df):
    df['연월일'] = pd.to_datetime(df['연월일'])
    df['시간'] = df['시간'].astype(int)
    df = df.apply(change_dates_adversely, axis=1)
    
    del df['datetime']
    
    return df


path = '/Users/younghun/Desktop/gitrepo/dacon_gas'
gas_weather = merge_gas_weather(path)
dataset = make_datetime_vars(gas_weather)
dataset = extract_need_data(dataset)
final_df = fe_autocorr_vars(dataset)
final_df = change_date_format(final_df)
final_df = final_df.reset_index(drop=True)
print(final_df.shape)